In [7]:
!pip install -U vllm --pre --extra-index-url https://wheels.vllm.ai/nightly



Looking in indexes: https://pypi.org/simple, https://wheels.vllm.ai/nightly
  Using cached vllm-0.12.0.tar.gz (17.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cachetools-6.2.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached tiktoken-0.12.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached ninja-1.13.0-py3-none-macosx_10_9_universal2.whl.metadata (5.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)


Import

In [8]:
from vllm import LLM, SamplingParams
from vllm.model_executor.models.deepseek_ocr import NGramPerReqLogitsProcessor
from PIL import Image
import pandas as pd
from tqdm import tqdm
import os
from jiwer import cer, wer


/Users/lehoangan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping import of cpp extensions due to incompatible torch version 2.9.1 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1212 16:20:55.432000 59216 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


INFO 12-12 16:20:56 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.


/Users/lehoangan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:646: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


Load llm

In [9]:
llm = LLM(
    model="unsloth/DeepSeek-OCR",   # <-- THIS IS THE BASELINE
    enable_prefix_caching=False,
    mm_processor_cache_gb=0,
    logits_processors=[NGramPerReqLogitsProcessor]
)


INFO 12-12 16:21:07 [utils.py:253] non-default args: {'seed': None, 'enable_prefix_caching': False, 'disable_log_stats': True, 'mm_processor_cache_gb': 0, 'logits_processors': [<class 'vllm.model_executor.models.deepseek_ocr.NGramPerReqLogitsProcessor'>], 'model': 'unsloth/DeepSeek-OCR'}


WARNING 12-12 16:21:08 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.
INFO 12-12 16:21:16 [model.py:637] Resolved architecture: DeepseekOCRForCausalLM
INFO 12-12 16:21:16 [model.py:1750] Using max model len 8192
WARNING 12-12 16:21:16 [cpu.py:152] Environment variable VLLM_CPU_KVCACHE_SPACE (GiB) for CPU backend is not set, using 4 by default.
INFO 12-12 16:21:16 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=4096.


Skipping import of cpp extensions due to incompatible torch version 2.9.1 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1212 16:21:25.084000 62913 myenv/lib/python3.11/site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


INFO 12-12 16:21:25 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.


/Users/lehoangan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:646: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


(EngineCore_DP0 pid=62913) INFO 12-12 16:21:26 [core.py:93] Initializing a V1 LLM engine (v0.12.0) with config: model='unsloth/DeepSeek-OCR', speculative_config=None, tokenizer='unsloth/DeepSeek-OCR', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cpu, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None, kv_cache_metrics=False, kv_cache_metrics_sample=0.01), 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:17<00:00, 17.24s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:17<00:00, 17.24s/it]
(EngineCore_DP0 pid=62913) 


(EngineCore_DP0 pid=62913) INFO 12-12 16:25:34 [default_loader.py:308] Loading weights took 17.25 seconds
(EngineCore_DP0 pid=62913) WARNING 12-12 16:25:34 [vllm.py:1322] Current vLLM config is not set.
(EngineCore_DP0 pid=62913) INFO 12-12 16:25:34 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=2048.
(EngineCore_DP0 pid=62913) INFO 12-12 16:25:34 [kv_cache_utils.py:1286] GPU KV cache size: 69,888 tokens
(EngineCore_DP0 pid=62913) INFO 12-12 16:25:34 [kv_cache_utils.py:1291] Maximum concurrency for 8,192 tokens per request: 8.53x
(EngineCore_DP0 pid=62913) INFO 12-12 16:25:34 [cpu_model_runner.py:65] Warming up model for the compilation...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

(EngineCore_DP0 pid=62913) INFO 12-12 16:25:56 [cpu_model_runner.py:75] Warming up done.
(EngineCore_DP0 pid=62913) INFO 12-12 16:25:56 [core.py:254] init engine (profile, create kv cache, warmup model) took 21.79 seconds
(EngineCore_DP0 pid=62913) WARNING 12-12 16:25:57 [vllm.py:608] Inductor compilation was disabled by user settings,Optimizations settings that are only active duringInductor compilation will be ignored.
INFO 12-12 16:25:57 [llm.py:343] Supported tasks: ['generate']


Sampling paramters

In [10]:
sampling_param = SamplingParams(
    temperature=0.0,
    max_tokens=8192,
    extra_args=dict(
        ngram_size=30,
        window_size=90,
        whitelist_token_ids={128821, 128822},  # <td> and </td>
    ),
    skip_special_tokens=False,
)


Run baseline

In [ ]:
IMG_ROOT = "dataset/images"
TEST_CSV = "dataset/test.csv"
PROMPT = "<image>\nFree OCR."    # same prompt as notebook baseline

df = pd.read_csv(TEST_CSV)

preds = []
gts = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    rel = row["filepath"]
    gt = str(row["text"])
    img_path = os.path.join(IMG_ROOT, rel)

    image = Image.open(img_path).convert("RGB")

    model_input = [{
        "prompt": PROMPT,
        "multi_modal_data": {"image": image}
    }]

    out = llm.generate(model_input, sampling_param)
    pred = out[0].outputs[0].text

    preds.append(pred)
    gts.append(gt)

print("CER:", cer(gts, preds))
print("WER:", wer(gts, preds))


Adding requests: 100%|██████████| 1/1 [00:07<00:00,  7.27s/it]
